<a href="https://colab.research.google.com/github/toraaglobal/RemoteSensing/blob/master/Land_Use_Classification_for_Nigeria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Land Use Classification for Nigeria

In [1]:
import ee
import geemap
import folium

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='toraaglobal')

In [3]:
# Define Nigeria boundary using the 'FAO GAUL' dataset
nigeria = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Nigeria')) \
            .geometry()

In [4]:
# Load and filter Sentinel-2 imagery (2022 dry season)
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(nigeria) \
    .filterDate("2022-11-01", "2023-03-31") \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda img: img.divide(10000)) \
    .median() \
    .clip(nigeria)

In [5]:
# Select RGB and NIR bands
bands = ['B2', 'B3', 'B4', 'B8']  # Blue, Green, Red, NIR


In [6]:
# Define training points (manual sampling or known labeled data)
# For demonstration, we're using synthetic geometries and labels
points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([7.5, 9.0]), {'landcover': 0}),  # Water
    ee.Feature(ee.Geometry.Point([7.0, 10.5]), {'landcover': 1}), # Forest
    ee.Feature(ee.Geometry.Point([8.5, 11.0]), {'landcover': 2}), # Urban
    ee.Feature(ee.Geometry.Point([6.5, 8.5]), {'landcover': 3}),  # Agriculture
    ee.Feature(ee.Geometry.Point([5.5, 9.5]), {'landcover': 4})   # Grassland
])

In [7]:
# Sample the image at training point locations
training = sentinel2.select(bands).sampleRegions(
    collection=points,
    properties=['landcover'],
    scale=10
)

In [8]:
# Train a classifier (Random Forest)
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=training,
    classProperty='landcover',
    inputProperties=bands
)

In [9]:
# Apply the classifier to the image
classified = sentinel2.select(bands).classify(classifier)

In [10]:
# Define visualization parameters
landcover_palette = ['0000ff', '008000', 'ff0000', 'ffff00', '00ffff']
landcover_names = ['Water', 'Forest', 'Urban', 'Agriculture', 'Grassland']
viz_params = {'min': 0, 'max': 4, 'palette': landcover_palette}

In [11]:
# Display using geemap
Map = geemap.Map(center=[9.1, 8.7], zoom=6)
Map.addLayer(classified, viz_params, "Land Use Classification")
Map.add_legend(title="Land Cover", labels=landcover_names, colors=landcover_palette)
Map.addLayer(nigeria, {}, "Nigeria Boundary")
Map

Map(center=[9.1, 8.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…